In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from mongo import attraction_db
from model.google import Google
from model.Word import Word
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
import random
import os
import json
import time
import csv
import os

c:\2_DataScience\04_google_map_crawler\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
url = 'https://www.google.com/maps/search/%E6%99%AF%E9%BB%9E'

driver = webdriver.Chrome()
driver.get(url)

In [ ]:
def getPage():
    page_content = driver.page_source
    # 將 HTML 內容轉換成 BeautifulSoup 物件，html.parser 為使用的解析器
    soup = BeautifulSoup(page_content, 'html.parser')
    # 透過 select 使用 CSS 選擇器 選取我們要選的 html 內容
    return soup.select('.Nv2PK.THOPZb.CpccDe')

def scrollPage(times):   # 滾動頁面
    counter = 0
    while counter <= times:
        pane = driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
        counter += 1

def getElement():  # 抓取目前的所有留言
    spots_list = []
    scrollPage(5)
    elements = getPage()

    for element in elements:
        name = element.select('.qBF1Pd.fontHeadlineSmall')[0].text
        url = element.select('.hfpxzc')[0].get('href')
        dataset = {}
        dataset['name'] = name
        dataset['google_url'] = url

        spots_list.append(dataset)
    return spots_list

In [ ]:
lastSpot = ''
while 7 == 7:
    print(lastSpot)
    spots_list = getElement()
    if (spots_list[len(spots_list)-1]["name"] == lastSpot ):
        break
    lastSpot = spots_list[len(spots_list)-1]["name"]
    time.sleep(1)
    getElement()

csv_file = './data/spot_list/spots.csv'
if os.path.exists(csv_file):
    with open(csv_file, mode="r", newline="", encoding="utf-8-sig") as file:
        reader = csv.DictReader(file)
        existing_data = list(reader)
else:
    existing_data = []

for spot in spots_list:
    existing_data.append(spot)

print(len(existing_data))


with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as file:
    # 定義 CSV 欄位
    fieldnames = ["name", "google_url"]
    # 建立 CSV 寫入器
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

    print(f"已將資料寫入 CSV 檔案：{csv_file}")

In [ ]:
dataset = pd.read_csv('./data/spot_list/spotsList_clean.csv')
dataset

In [ ]:
df_unique = dataset.drop_duplicates(subset="name", keep="first")
df_unique.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")
df_unique

In [ ]:
existing_names = attraction_db.attractionInfo.distinct("name")
df_filtered = df_unique[~df_unique["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")

In [ ]:
existing_names = attraction_db.attractionInfo.distinct("name")
df = pd.read_csv('./data/spot_list/random_spotsList.csv')
df_filtered = df[~df["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList_clean.csv', index=False, encoding="utf-8-sig")
df_filtered

In [ ]:
# ============================================
# TEST
# ============================================

spotsList = pd.read_csv('./data/spot_list/spotsList_clean.csv') 

for index, row in spotsList[558:570].iterrows():
    print(index,row['name'],row['google_url'])

In [ ]:
# 取得 Google URL

name_to_url = {}
with open('./data/spot_list/spotsList_clean.csv', 'r', encoding='utf-8') as spots_file:
    spots_reader = csv.reader(spots_file)
    next(spots_reader)  # 跳过标题行
    for row in spots_reader:
        name = row[0]  # 假设名称在第一列
        url = row[1]   # 假设URL在第二列
        name_to_url[name] = url

def get_url_for_name(name):
    return name_to_url.get(name, "URL not found")

In [18]:
# 含有多少沒有抓

existing_names = attraction_db.attractionInfo.distinct("name")
df = pd.read_csv('./data/spot_list/spotsList_clean.csv')
df_filtered = df[~df["name"].isin(existing_names)]
df_filtered

,name,google_url
8,真笠山東南峰,https://www.google.com/maps/place/%E7%9C%9F%E7...
16,明池國家森林遊樂區,https://www.google.com/maps/place/%E6%98%8E%E6...
19,延平河濱公園,https://www.google.com/maps/place/%E5%BB%B6%E5...
31,六十石山金針花海,https://www.google.com/maps/place/%E5%85%AD%E5...
37,鴉片菸潭,https://www.google.com/maps/place/%E9%B4%89%E7...
...,...,...
974,白布帆砲台古道,https://www.google.com/maps/place/%E7%99%BD%E5...
975,都蘭文創園區,https://www.google.com/maps/place/%E9%83%BD%E8...
976,三義木雕博物館,https://www.google.com/maps/place/%E4%B8%89%E7...
979,大園華興池公園（華興埤/6-12號埤塘）,https://www.google.com/maps/place/%E5%A4%A7%E5...


In [ ]:
# ============================================
# 個別處理
# ============================================

name = 'wind wall'
url  = get_url_for_name(name)
try:
    data = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
    tag_value = data['words'].tolist()
except:
    tag_value = []

place_detail_data = Google().get_place_info(name,url,tag_value)
place_detail_data = json.loads(place_detail_data)
print(place_detail_data)

existing_data = attraction_db.attractionInfo.find_one({
    "$or": [
        {"place_id": place_detail_data["place_id"]},
        {"id": place_detail_data["id"]}
    ]
})
print(existing_data)
if existing_data is None:
    attraction_db.attractionInfo.insert_one(place_detail_data)
    print(f"{place_detail_data['name']}：已新增")

In [17]:
# 檢查未成功插入資料庫的景點

existing_names = set(doc['name'] for doc in attraction_db.attractionInfo.find({}, {"name": 1}))

csv_folder_path = './data/tfidf_csv/' 
csv_files = os.listdir(csv_folder_path)
new_csv_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
print(len(new_csv_files))
print(new_csv_files)

0
[]


In [12]:
# 新增缺少的景點
wrong = []
# new_csv_files = ['三峽螢火蟲秘境']
for spot in new_csv_files:
    name = spot
    url = get_url_for_name(name)
    try:
        try:
            tfidf = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
            tag_value = tfidf['words'].tolist()
        except:
            tag_value = []
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            wrong.append({
                "name":place_detail_data['name'],
                "existing_data":existing_data['name']
            })
            print(f"{place_detail_data['name']}：未新增",existing_data['name'])
    except Exception as e:
        wrong.append({
            "name":name,
            "existing_data":e
        })
        print(f"錯誤訊息：{e}")
        continue

print(wrong)
with open("./data/warn/check.json", "w", encoding="utf8") as f:
    json.dump(wrong, f, indent=2, sort_keys=True, ensure_ascii=False)

Graffiti：已新增
Laiyi Condor Waterfall：未新增 望古瀑布
Shuixing Suspension Bridge：未新增 Keelung beach
Songyuan Villa：已新增
【開放預約試營運！】愛玩色創意館 顏料觀光工廠｜親子旅遊｜彰化必玩｜水性無毒顏料｜DIY課程：已新增
三和瓦窯：已新增
三義木雕博物館：未新增 木雕博物館
不要臉公園：已新增
世紀大峽谷：已新增
中央尖：已新增
中正紀念林：已新增
中港河廊：已新增
二峰圳：已新增
二重疏洪運動公園：已新增
仙跡岩步道：已新增
倒影塔：已新增
優質海景下去秘道：已新增
光隆博物館：已新增
六十石山風景區：已新增
冬山河伯朗大道：已新增
出磺坑古道：已新增
利澤簡老街：已新增
加拿瀑布：已新增
勇士廣場：已新增
卑南遺址考古現場：已新增
南崁兒童藝術村：已新增
南投日月潭：已新增
南投縣文學步道：已新增
南投龍鳳瀑布：已新增
古崗樓：已新增
台南二寮觀日亭：已新增
台灣原住民族文化園區：未新增 臺灣原住民文化園區
台灣原住民族文化館：已新增
台灣小瑞士：已新增
右岸濕地：已新增
同源橋：已新增
和平礁：未新增 Keelung beach
嘉德花田：已新增
城前鮮蚵觀光碼頭：已新增
埔頂大草原：已新增
埤頭木棉花道：已新增
墾丁大街：已新增
大安海灘：已新增
大寮茶文館：已新增
大目降廣場：已新增
大目降故事館：已新增
大碗公溜滑梯：已新增
太平洋玫瑰莊園：已新增
奇萊鼻燈塔：已新增
奶奶的熊毛巾故事館：已新增
安平運河公園：已新增
安永心食館：已新增
富興工廠1962：已新增
富谷灣：已新增
寧濟御庭園林山水文化園區：已新增
小南海風景區：已新增
小泰山森林浴步道：已新增
山谷林野生食用植物園：已新增
布袋濕地生態園區：已新增
彩虹廊道：已新增
後壁湖潮間帶：已新增
後浦16藝文特區：已新增
忘憂谷：已新增
恐龍園區公園：已新增
成功沙灘：已新增
文筆山觀景亭：已新增
新埔海堤：已新增
新娘婚紗瀑布：已新增
新屋區海洋客家休閒農業區遊客中心：已新增
旗津星空隧道：已新增
日軍臺灣步兵第二聯隊營舍舊址：已新增
明池：未新增 明池森林遊樂區
朴子藝術公園：已新增
枋寮老街：已新增
林投公園：已新增
栗松溫泉：已新增
桃園機場 05R 跑道頭觀賞點（大園埔心里）：已新增
梧棲老

In [16]:
# 刪除不必要的景點
for spot in new_csv_files:
    name = spot
    csv_file_path = './data/tfidf_csv/{}.csv'.format(name)
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
        print(f"{name}.csv 已删除")

Laiyi Condor Waterfall.csv 已删除
Shuixing Suspension Bridge.csv 已删除
三義木雕博物館.csv 已删除
台灣原住民族文化園區.csv 已删除
和平礁.csv 已删除
明池.csv 已删除
水火同源.csv 已删除
汀甫圳步道.csv 已删除
海天一線看海區.csv 已删除
王功風車.csv 已删除
白布帆砲台古道.csv 已删除
西門町徒步區.csv 已删除
都蘭文創園區.csv 已删除
金崙溫泉.csv 已删除
霄裡大池落羽松林.csv 已删除
霞喀羅瀑布.csv 已删除
鯉魚潭觀景台.csv 已删除


In [ ]:
# 隨機排序景點

list = pd.read_csv('./data/spot_list/spotsList.csv')
print(list)

random.seed(42)
random_spotsList = list.sample(frac=1,random_state=random.randint(1,10000))
print(random_spotsList)

random_spotsList.to_csv('./data/spot_list/random_spotsList.csv',encoding="utf-8-sig",index=False)


In [ ]:
# ============================================
# 個別處理 2
# ============================================
ws_driver = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")

In [ ]:
csv_folder_path = './data/set/' 
existing_names = attraction_db.attractionInfo.distinct("name")
csv_files = os.listdir(csv_folder_path)
tfidf_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
tfidf_files

In [ ]:
for spot in tfidf_files:
    name = spot
    url = get_url_for_name(name)
    try:
        dataset = pd.read_excel('./data/set/{}.xlsx'.format(name))
        tag = Word(ws_driver,pos_driver).save(dataset)
        tag_value = tag['words'].tolist()
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            print(f"{place_detail_data['name']}：未新增")
    except Exception as e:
        print(f"錯誤訊息：{e}")
        continue